<a href="https://colab.research.google.com/github/Anze-/datathon2k25/blob/alberto/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

# 1. Set up working environment

In [1]:
# enable GPU if needed, GPU can speed up your vector embedding if you computing these vectors locally (not using API)

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [30]:
import os
import json
import chromadb
import openai
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Set OpenAI API Key
os.environ["OPENAI_API_KEY"] = open('api.key').read()[:-1]

# 2. Knowledge Base Preparation

## 2.1 Load documents

Once you are added access to this folder, it will appear at your google drive "Shared drives". Then you can mount your drive and as following, and access your data from "/content/drive/Shared drives/Datathon/Data/hackathon_data/". Enjoy the ride! :)

In [32]:
folder_path = "./data/hackathon_data/"# Google drive path of the dataset
files_in_folder = os.listdir(folder_path)

len(files_in_folder)

13144

In [33]:
def load_documents(json_file):
    """Loads the JSON file."""
    with open(json_file, 'r') as f:
      try:
          data = json.load(f)
          return data
      except json.JSONDecodeError:
          print(f"Error reading {json_file}, it may not be a valid JSON file.")
    return []

In [9]:
files_in_folder

['skysolutions.com.json',
 'richardsonsports.com.json',
 'wilson-company.com.json',
 'westernallied.com.json',
 'bluescopebuildings.com.json',
 'cobbemc.com.json',
 'is4s.com.json',
 'berkleyselect.com.json',
 'webbwheel.com.json',
 'sygmanetwork.com.json',
 'smma.com.json',
 'cgicontainersales.com.json',
 'nydig.com.json',
 'protranslating.com.json',
 'traditionalbank.com.json',
 'percona.com.json',
 'lawnlove.com.json',
 'elementsdesign.com.json',
 'pghwong.com.json',
 'fwcook.com.json',
 'sofistadium.com.json',
 'sfopera.com.json',
 'helenwellsagency.com.json',
 'russdaviswholesale.com.json',
 'irissoftware.com.json',
 'klarquist.com.json',
 'govtact.com.json',
 'lowlandsgroup.com.json',
 'kingseducation.com.json',
 'qualcareinc.com.json',
 'mymotomart.com.json',
 'chartbeat.com.json',
 'kongbasileconsulting.com.json',
 'javacity.com.json',
 'saintmarks.com.json',
 'finchpaper.com.json',
 'optoro.com.json',
 'paccarparts.com.json',
 'tendercarehh.com.json',
 'ftei.com.json',
 'lesse

In [8]:
for filename in files_in_folder:
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        doc = load_documents(file_path)
        break
print(doc.keys())
doc

dict_keys(['url', 'timestamp', 'text_by_page_url', 'doc_id'])


{'url': 'http://skysolutions.com',
 'timestamp': 1734917198.7669182,
 'text_by_page_url': {'http://skysolutions.com': "Empowering Federal Clients with AI-Powered Digital Solutions - Sky Solutions\nSkip to content\nServices\nAI Solutions\nEnterprise Data Management and\u200b Platform Enablement\u200b\nAI/ML and Advanced Analytics\nGenerative AI Platforms and Solutions\u200b\nDigital Transformation\nHuman-Centered Design\nAgile/SAFe & Agile Product Management\nCI/CD and DevSecOps Practices\nLow-Code / No-Code Development\nLegacy Modernization and Cloud Migration\nCybersecurity\nZero Trust Architecture\u200b\nRisk Management Framework\nApplication and Cloud Security Engineering\nSecurity Operations\nAI Solutions\nEnterprise Data Management and\u200b Platform Enablement\u200b\nAI/ML and Advanced Analytics\nGenerative AI Platforms and Solutions\u200b\nDigital Transformation\nHuman-Centered Design\nAgile/SAFe & Agile Product Management\nCI/CD and DevSecOps Practices\nLow-Code / No-Code Devel

## 2.2 Pre-process documents.

Feel free to explore and pre-process the data. You may want to clean or segment the documents as you see fit.

In [ ]:
def page_segment(docs):
    """You may prefer to load each page separately."""
    i = 0
    page_segment = []
    for s in list(docs['text_by_page_url'].values()):
      page_segment.append({"docID": docs['doc_id'], "pageID": 'page_' + str(i), "text": s})
      i += 1
    return page_segment

In [ ]:
def segment_documents(docs, chunk_size=500):
    """Segments documents into chunks of a given token size. Replace this function with your segmentation approach or maybe use the original document without segmentation."""
    segmented = []
    for doc_id, content in docs.items():
        for i in range(0, len(content), chunk_size):
            segment = content[i : i + chunk_size]
            segmented.append({"id": doc_id, "text": segment})
    return segmented



In [ ]:
def document_clean(docs):
  """
  You may want to clean the dataset, add the code here.
  """
  pass

## 2.3 Document Indexing and Storage (Profiling)

Feel free to choose different ways to indexing and storing the provided documents in a knowledge database.

So that they can be retrieved in different ways according to your system design choices, such as search by keywords, vector representation, graph relation, and etc.

# 3. Retrieval Augmented Generation

## 3.1 Load Knowledge Database

## 3.2 Relevant Document Retrieval

Feel free to check and improve your retrieval performance as it affect the generation results significantly.

In [ ]:
def retrieve_documents(query, db_path, embedding_model):
  """
  retrieve relevant documents from the knowledge database to the query.
  """
  return relevant_docs

## 3.3 Response Generation

Feel free to explore promp engineer to improve the quality of your generated response.

The retrieved documents are used as context to generate more relevant response. Gereral knowledge from the language model itself is also used.

In [ ]:
def generate_answer(query, retrieved_texts, prompt_template):
    """Generates an answer using retrieved documents and GPT-4."""
    return response

In [ ]:
query = "What company is located in 29010 Commerce Center Dr., Valencia, 91355, California, US?"
retrieved_docs = retrieve_documents(query, db_path, embedding_model)
response = generate_answer(query, retrieved_texts, prompt_template)

print("Query:", query)
print("Retrieved Documents:", [doc.page_content for doc in retrieved_docs])
print("Generated Answer:", response)

# 4. Evaluation

Try as many examples to evaluate your system and improve your performance!

As the final sysrtem will be evaluated from various aspects. Try to check different metrics when you evaluate. One trick is to do a "strict RAG" where the response is generated based on the retrieved documents only, i.e. no general knowledge from the LLMs will be used. This may be a good way to check if your retrieval part is working as expected. Note, that in the final system general knowledge from the LLMs are welcome. "Strict RAG" is only used as a way for you to check your performance :)